# Computational Theory Problems

In [111]:
import numpy as np 
import math

## Introduction




## Problem 1: Binary Words and Operations

### SHA-1 Functions
SHA-1 runs 80 rounds of bitwise operations, using three logical functions — Parity, Ch, and Maj — to mix three 32-bit words (x, y, z) in each round.

The algorithm’s output is a 160-bit hash value, represented as five 32-bit words.<br><br>
Every rotation, XOR, or addition must stay within 32 bits — if anything overflows, <br>
`np.uint32` wraps it automatically, keeping the math identical to real 32-bit hardware.

In [112]:
#Assigning variables as 32 bit unsigned integers
x = np.uint32
y = np.uint32
z = np.uint32

### Parity Function

**Formula:** Parity(x XOR y XOR z)

**Purpose:** This function mixes three 32-bit numbers (x, y, and z) using XOR to add variation to the hash.

**What function is doing:**
1.  **x XOR y**:<br>
        Compares every bit of x to every bit of y.<br>
        If bits differ, result is 1; otherwise 0.

2.  **(x XOR y) XOR z**:<br>
        Takes the result of step 1 and compares it bit-by-bit with *z*.<br>
        If bits differ, result is 1; otherwise 0.<br>
        (Effectively counts whether the total number of 1 bits across x, y, z is odd.)



In [113]:
def Parity(x, y, z):

    """Computes the SHA-1 Parity function."""
    
    #Returns result of parity function as 32-bit unsigned integer
    return (x ^ y ^ z)

#Testing function 
result = Parity(14, 6, 1)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form


Binary result: 00000000000000000000000000001001
Decimal result: 9


### Ch(Choose) Function
 **Formula:** Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z)

 **Purpose:** This function picks bits from y or z depending on the value of x, helping to mix data in each round.

**What function is doing:**
1.  **x & y**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

2.  **~x & z**:<br>
Flips all bits of x into opposite bits.<br>
Compares every bit of x to every bit of z.<br>
If both bits are 1, the result is 1, otherwise 0.

3.  **XOR**:<br> 
Compares Step 1 against Step 2 bit by bit.<br>
If bits differ, result is 1; otherwise 0.

In [114]:
def Ch(x, y, z):

    """Computes the SHA-1 Ch (choose) function"""

    #Returns result of Ch function as 32 bit unsigned integer
    return (x & y) ^ (~x & z)

#Testing function 
result = Ch(13, 6, 8)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00000000000000000000000000000100
Decimal result: 4


### Maj Function
 **Formula:** Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)

 **Purpose:** This function finds the most common bit among x, y, and z to help SHA-1 mix values fairly.

**What function is doing:**
1.  **x & y**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

2.  **x & z**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

3.  **y & z**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

4. **XOR**:<br>
Compares the three results from Steps 1–3 bit by bit.<br>
If bits differ, result is 1; otherwise 0.

In [115]:
def Maj(x, y, z):

    """Computes the SHA-1 Maj function"""

    #Returns result of maj function as 32 bit unsigned integer
    return ((x & y) ^ (x & z) ^ (y & z))

#Testing function 
result = Maj(19, 21, 4)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00000000000000000000000000010101
Decimal result: 21


###  SHA-224 and SHA-256 Functions
SHA-224 and SHA-256 use the same logical building blocks as SHA-1, but operate within the SHA-2 family of hash algorithms.<br><br>
Unlike SHA-1, which runs 80 rounds, SHA-224 and SHA-256 each run 64 rounds,<br>
using a larger 256-bit state and more complex logical functions for stronger security.<br><br>

It uses a mix of 6 functions - Ch, Maj, Σ₀, Σ₁, σ₀, and σ₁.<br><br>

Instead of five 32-bit words (like SHA-1’s 160-bit state), SHA-2 works with eight 32-bit words,<br>giving outputs of 224 or 256 bits. ([SHA-224 and SHA-256 explanation](https://mojoauth.com/compare-hashing-algorithms/sha-224-vs-sha-256/))

### Rotate Right Function

**Formula:** ROTR^n(x) = (x >> n) OR (x << (w - n))<br>
Where w = 32 for a 32-bit word


**Purpose:** This `rotr` function ([adapted from GeeksforGeeks](https://www.geeksforgeeks.org/dsa/rotate-bits-of-an-integer/)) performs a circular right shift within a fixed 32-bit word,<br>
moving the bits that fall off the right back to the left side.

**What function is doing:**<br>
**n = n % 32:**<br>
Ensures the rotation amount stays between 0–31.<br>
Rotating by 32 (or any multiple) returns the number unchanged.<br>
Eg: If n = 35, then n % 32 = 3 - rotate by 3 bits.<br>

1. **x >> n**:<br>
        Shifts every bit of x to the right by n positions.<br>
        The rightmost bits are dropped and zeros fill in on the left.<br>


2. **32 - n**:<br>
        Represents the number of bits that need to wrap around from the right side to the left.<br>

3. **x << (32 - n)**:
        Moves the dropped bits back to the left side of the 32-bit word, maintaining circular rotation<br>

4. **OR:**<br>
        Compares results from Step 1 and Step 3 bit by bit.
        If either of the bits are 1, result is bit is 1; Otherwise its 0


        



In [116]:
def rotr(x, n):

    """Performs a 32-bit circular right rotation"""

    #Keep rotation within 32 bit word range
    n = n % 32

    #Shift bits right by n and move lost bits to left side
    rotr = ((x >> n) | (x << (32 - n))) 
    

    #Returning result as 32 bit unsigned integer
    return rotr

#Testing function 
result = rotr(12, 2)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 1100000000000000000000000000000011
Decimal result: 12884901891


### Rotate Shift Function

**Formula:** SHR^n(x) = (x >> n) <br>

**Purpose:** This `rightShift` function performs a right shift operation by moving bits of x to the right by n positions,<br>
filling in the left side with zeros.

**What function is doing:**<br>

1. **n = shift amount (0–31):**<br>
Specifies how many positions the bits in x will move to the right.  

2. **x >> n:**<br>
Shifts every bit of x right by n positions.  
The rightmost bits are killed, and zeros fill in on the left.  

3. **n32(x >> n):**<br>
Ensures the result stays within a 32-bit unsigned integer range.   



In [117]:
def rightShift(x, n):

    """Performs a right rotation shift operation"""
    
    #Shift bits right by n and adds 0's to the left side in replacement of shifted bits
    rightShift = (x >> n)

    #Returning result as 32-bit unsigned integer
    return rightShift

#Testing function 
result = rightShift(58, 2)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00000000000000000000000000001110
Decimal result: 14


### Σ₀ (Sigma0) Function

**Formula:** Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)<br>

**Purpose:** This function rotates the bits of input x to the right by 2, 13, and 22 bits.<br>
It increases mixing of bits to strengthen the hash’s randomness and security.

**What the function is doing:**<br>

1. **rotr(x, 2):**<br>
   Calls the `rotr` function.<br>
   Rotates all bits of *x* to the right by 2 bits.

2. **rotr(x, 13):**<br>
   Calls the `rotr` function.<br>
   Rotates all bits of *x* to the right by 13 bits.

3. **rotr(x, 22):**<br>
   Calls the `rotr` function.<br>
   Rotates all bits of *x* to the right by 22 bits.

4. **XOR:**<br>
   Combines the results from Steps 1–3 bit by bit.<br>
   If bits differ, result is 1; otherwise 0.


In [118]:
def Sigma0(x):

    """Computes the SHA-2 Σ₀(x) function."""

    #Returning result of right-rotating x by 2, 13, and 22 bits,
    #then XORing the three results together    
    return (rotr(x,2) ^ rotr(x, 13) ^ rotr(x, 22))

#Testing function 
result = Sigma0(12)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 1100000000011000000011000000000011
Decimal result: 12891205635


### Σ₁ (Sigma1) Function

**Formula:** Σ₁(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)

**Purpose:** This function works exactly like Σ₀(x) (see previous section),  
but uses different rotation amounts - 6, 11, and 25 bits —  
to increase the mixing of bits at a different stage of the hash computation.

**Reference:** Implementation adapted from the same pattern used in Σ₀(x),  
with only rotation values changed.

In [119]:
def Sigma1(x):

    """Computes the Σ₁(x) function."""

    #Returning result of right-rotating x by 6, 11, and 25 bits,
    #then XORing the three results together    
    return (rotr(x,6) ^ rotr(x, 11) ^ rotr(x, 25))

#Testing function 
result = Sigma1(12)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00110001100000000000011000000000
Decimal result: 830473728


### σ₀ (sigma0) Function

**Formula:** σ₀(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)

**Purpose:** This function rotates the bits of input x to the right by 7 and 18 bits.<br>
It then performs a right shift operation on x by 3 bits.
It increases mixing of bits to strengthen the hash’s overall security.

**What the function is doing:**<br>

1. **rotr(x, 7):**<br>
   Calls the `rotr` function.<br>
   Rotates all bits of x to the right by 7 bits.

2. **rotr(x, 18):**<br>
   Calls the `rotr` function.<br>
   Rotates all bits of x to the right by 18 bits.

3. **rightShift(x, 3):**<br>
   Calls the `rightShift` function.<br>
   Shifts all bits of x to the right by 3 bits, replacing leftmost bits with zeros.

4. **XOR:**<br>
   Combines the results from Steps 1–3 bit by bit.<br>
   If bits differ, result is 1; otherwise 0.

In [120]:
def sigma0(x):

    """Computes the σ₀(x) function."""

    #Returning result of right-rotating x by 7, and 18, and then
    #right-shifting x by 3
    return (rotr(x,7) ^ rotr(x, 18) ^ rightShift(x, 3))#XOR of 3 results

#Testing function 
result = sigma0(12)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00011000000000110000000000000001
Decimal result: 402849793


### σ₁ (sigma1) Function

**Formula:** σ₁(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)

**Purpose:** This function works exactly like σ₀(x) (see previous section),  
but uses different rotation and right shift amounts - 17, 19, and 10 bits —  
to increase the mixing of bits at a different stage of the hash computation.

**Reference:** Implementation adapted from the same pattern used in σ₀(x),  
with only rotation and right shift values changed.

In [121]:
def sigma1(x):

    """Computes the σ₁(x) function."""

    #Returning result of right-rotating x by 17, and 19, and then
    #right-shifting x by 10
    return (rotr(x,17) ^ rotr(x, 19) ^ rightShift(x, 10))#XOR of 3 results

#Testing function 
result = sigma1(12)
print(f"Binary result: {result:032b}") #Binary output as 32-bit word
print(f"Decimal result: {result}") #Result in decimal form

Binary result: 00000000000001111000000000000000
Decimal result: 491520


## Problem 2: Fractional Parts of Cube Roots

### Find Prime Function

**The Sieve of Eratosthenes Algorithm**<br>
The Sieve of Eratosthenes Algorithm is an efficient algorithm for finding all prime numbers up to a given limit.<br>
My understanding of the algorithm was developed through the explanation provided in ([LibreTexts](https://math.libretexts.org/Courses/Coalinga_College/Math_for_Educators_(MATH_010A_and_010B_CID120)/04%3A_Number_Theory/4.03%3A_The_Sieve_of_Eratosthenes)), which illustrates the process of iteratively crossing out multiples of each prime starting from 2.
The method continues until all non-prime numbers have been marked, leaving only prime numbers unmarked.


**Purpose:** The `find_primes` function ([adapted from GeeksforGeeks](https://www.geeksforgeeks.org/python/python-program-for-sieve-of-eratosthenes/)), finds the first `n` prime numbers using a vectorized version of the Sieve of Eratosthenes algorithm.<br><br>
I decided to use this method of finding prime numbers as it is a fast and memory efficient way of generating prime numbers, particularly as the number of required primes increases.<br><br>
When testing prime functions from
([given documentation](https://github.com/ianmcloughlin/computational-theory/blob/main/materials/prime_numbers.ipynb)), on larger numbers such as 90000, the time taken to return the primes was slower (6s), compared to vectorised Sieve Algorithm (0s).<br><br>
The algorithm systematically eliminates non-prime numbers by marking multiples of each prime, and when implemented with ([NumPy vectorization](https://www.geeksforgeeks.org/numpy/vectorized-operations-in-numpy/)), this marking is performed in a single optimized operation rather than in repeated Python loops, leveraging NumPy's underlying C implementation for faster and more efficient computations.

**What the function is doing:**<br><br>
1. **prime = np.ones(k + 1, dtype=bool):**<br>
Creates a boolean array filled with `True` values, to mark any potential primes.

2. **prime[0:2] = False:**<br>
Uses ([slice function](https://www.geeksforgeeks.org/python/python-slice-function/)) to set 0 and 1 to False as they are not primes.

3. **for p in range(2, int(math.sqrt(k)) + 1):**<br>
Loops through every number from 2 up to square root of `k`. Any non-prime number will have at least one factor less than or equal to its square root, so checking beyond this point is unnecessary.
This means the loop only needs to test possible divisors up to the square root of k for efficiency.

4. **if prime[p]:**<br>
Checks if number in at current index is `True` - is prime.

5. **prime[p*p:k+1:p] = False**<br>
Start from the square root of `p`.<br>
Mark every multiple of `p` up to `k` as `False` (not prime).<br>
Do in steps of `p`.

6. **result = np.nonzero(prime)[0].tolist()**<br>
Returns indices of all non 0 - `True` values, i.e the primes and returns it in list.

7. **return result[:n]**<br>
Uses list slicing to return all prime numbers from 0 to n from the list.


In [ ]:

def find_primes(n):

    """Computes the first n primes numbers using vectorized version of Sieve of Eratosthenes"""
    #Upper limit large enough to find the first n primes
    k = 100000
    #Creates array filled with 1's (1 meaning True), Using a boolean array
    prime = np.ones(k + 1, dtype=bool)
    #Setting 0 and 1 to false, they are not primes by definition
    prime[0:2] = False

    #Loops through numbers from 2 to square root of n
    for p in range(2, int(math.sqrt(k)) + 1):
        #If current index is prime
        if prime[p]:
            prime[p*p:k+1:p] = False #Set its multiples to false
    #Returns all primes
    result = np.nonzero(prime)[0].tolist()
    #Gets all primes that were found only up to n
    return result[:n]

#Finding first 64 prime numbers
primes = find_primes(64)

#Printing prime numbers and how many there are
print(primes)       
print(len(primes))  


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
64


## Problem 3: Padding


## Problem 4: Hashes


## Problem 5: Passwords
